In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def delta(a, b, c):
  ''' Funcao para calcular o delta de uma equacao do segundo grau
  '''
  return (b**2)-4*a*c

def x(a, b, c):
  ''' Funcao para calcular as raizes de uma equacao do segundo grau
  '''
  # delta
  D = delta(a, b, c)
  if D < 0:
    return gr.update(value='No real roots', visible=True), gr.update(value='', visible=False), gr.update(value='', visible=False)

  # raiz com delta positivo
  x1 = (-b + D**(1/2))/(2*a)

  if D == 0:
    return gr.update(value='Single root', visible=True), gr.update(value=x1,visible=True), gr.update(value='', visible=False)

  # raiz com delta negativo
  x2 = (-b - D**(1/2))/(2*a)

  return gr.update(value='Two roots', visible=True), gr.update(value=x1, visible=True), gr.update(value=x2, visible=True)

def plot(a, b, c, x1, x2):
  '''Funcao para plotar o grafico da funcao quadradica dada por
  ax**2+bx+c com raizes x1 e x2
  As raizes, caso existam, sao destacadas como pontos vermelhos
  '''
  if x2 is not None:
    space = np.linspace(x1-10,x2+10,500)
  else:
    space = np.linspace(x1-10,x1+10,500)

  data = [a*i**2+b*i+c for i in space]

  fig, ax = plt.subplots()
  plt.plot(space,data)
  if x1 is not None:
    plt.plot(x1,0,'ro')
  if x2 is not None:
    plt.plot(x2,0,'ro')
  plt.plot(space,space*0,'k-')
  plt.plot(space*0,np.linspace(min(data)-5,max(data)+5,500),'k-')
  plt.grid()
  return fig

def solve_equation(equation):
  ''' Funcao para resolver uma equacao do segundo grau
  '''
  # pre-processamento: transformar equacao na forma:
  # Ax^2+Bx+C

  # removendo todos os espacos em branco
  equation = equation.strip()
  equation = equation.replace(' ', '')
  # transformando simbolo de potenciacao
  equation = equation.replace('**', '^')
  # removendo simbolo de multiplicacao
  equation = equation.replace('*', '')

  # se nao houver o termo quadratico, equacao esta incorreta
  if 'x^2' not in equation:
    return gr.update(visible=False),gr.update(visible=False),gr.update(visible=False),'Invalid equation',gr.update(visible=False),gr.update(visible=False),gr.update(visible=False)
  spl1 = equation.split('x^2')

  # avaliando valor de A
  if not spl1[0]:
    a = 1.
  elif spl1[0] == '-':
    a = -1.
  else:
    a = float(spl1[0])

  # avaliando valores de B e C
  spl2 = spl1[1].split('x')
  if len(spl2) == 1:
    b = 0.
    if not spl2[0]:
      c = 0.
    else:
      c = float(spl2[0])
  else:
    if spl2[0] == '-':
      b = -1.
    elif spl2[0] == '+':
      b = 1.
    else:
      b = float(spl2[0])
    if not spl2[1]:
      c = 0.
    else:
      c = float(spl2[1])
  print(a, b, c)

  message, x1, x2 = x(a, b, c)

  # plotando grafico apenas se pelo menos uma raiz
  if message['value'] == 'No real roots':
    grafico = gr.update(visible=False)
  elif message['value'] == 'Single root':
    grafico = gr.update(value=plot(a, b, c, float(x1['value']), None), visible=True)
  else:
    grafico = gr.update(value=plot(a, b, c, float(x1['value']), float(x2['value'])), visible=True)
  return gr.update(value=f'{a}', visible=True), gr.update(value=f'{b}', visible=True), gr.update(value=f'{c}', visible=True), message, x1, x2, grafico

# solve_equation("x**2+x+1")

In [ ]:
with gr.Blocks() as demo:
  output = []
  equation = gr.Textbox(label='Equation')
  solve_button = gr.Button('Solve')
  with gr.Row():
    A = gr.Number(label='A', visible=False)
    B = gr.Number(label='B', visible=False)
    C = gr.Number(label='C',visible=False)
  message = gr.Textbox(label="Message", visible=False)
  solution1 = gr.Textbox(label="Root #1", visible=False)
  solution2 = gr.Textbox(label="Root #2", visible=False)
  grafico = gr.Plot(label="Equation Plot", visible=False)
  solve_button.click(fn=solve_equation, inputs=equation, outputs=[A, B, C, message, solution1, solution2, grafico], api_name="2nd-solver")

demo.launch(debug=True)